In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
data = pd.read_csv("C:/Users/USER/Desktop/UCI_Credit_Card.csv (1)/UCI_Credit_Card.csv")
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import f1_score

In [4]:
data.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [45]:
data = data[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month']]
xdata = data[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']]
ydata = data['default.payment.next.month']
x_train , x_test , y_train , y_test = train_test_split(xdata , ydata)

## train on normal dataset

In [47]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

model = Sequential()
model.add(Dense(36, activation='relu', input_shape =(11,)))
model.add(Dropout(0.3))
model.add(Dense(36, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer='adam',loss = "categorical_crossentropy",metrics=['accuracy'])



y_train_one = np_utils.to_categorical(y_train)
y_test_one = np_utils.to_categorical(y_test)
#y_down_train_one =np_utils.to_categorical(y_down)
model.fit(x_train,y_train_one, epochs=10, validation_data=(x_test,y_test_one),class_weight=(1,10))
predict_one = model.predict(x_test)
predict_int = np.argmax(predict_one, axis=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 36)                432       
_________________________________________________________________
dropout_13 (Dropout)         (None, 36)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 36)                1332      
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 74        
Total params: 1,838
Trainable params: 1,838
Non-trainable params: 0
_________________________________________________________________
Train on 22500 samples, validate on 7500 samples
Epoch 1/10
22500/22500 [==============================] - 3s 113us/step - loss: 8.4914 - acc: 0.4732 - val_loss: 3.5202 - val_acc: 0.7816
Epoch 2/10
22500/22500 [==============================] - 2s 69us/step - loss: 3.6026

In [48]:
def score_matrix ( true , pred) :
    TP = np.sum(np.logical_and(pred==1,true==1))
    TN = np.sum(np.logical_and(pred==0,true==0))
    FP = np.sum(np.logical_and(pred==1,true==0))
    FN = np.sum(np.logical_and(pred==0,true==1))
    #判斷為真且為真的機率
    Precision = TP/(TP+FP)
    #為真被判斷為真的機率
    Recall = TP/(TP+FN)
    #為假被判斷為真的機率
    FPR = FP/(TN+FP)
    #平測不平衡資料的指標
    F1 = f1_score(y_true = true,y_pred=pred)
    ACC = accuracy_score (y_true=true,y_pred=pred)
    
    score = pd.DataFrame({"Index":["Precision","Recall","FPR","Accuracy","F1"],
                          "Score":[Precision,Recall,FPR,ACC, F1]})
    return score

In [49]:
pre = pd.DataFrame(predict_int)[0]
tru = pd.DataFrame(y_test.values)[0]

In [50]:
predict_int

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [51]:
score_matrix(tru,pre)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  import sys
D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,Index,Score
0,Precision,NaN
1,Recall,0.0000
2,FPR,0.0000
3,Accuracy,0.7816
4,F1,0.0000


## the result is pool , my model just predict it all zero to get 0.78 accuracy
## but the recall and precision are zero , it's totally useless

# train on down sampled dataset

In [52]:
from sklearn.utils import resample
train_set = x_train.join(y_train)
train_set["default.payment.next.month"].value_counts()

0    17502
1     4998
Name: default.payment.next.month, dtype: int64

In [53]:
## 把0的資料在訓練集上resameple降低到與1的資料相同數目
major = train_set[train_set["default.payment.next.month"]==0]
minor = train_set[train_set["default.payment.next.month"]==1]
major_down = resample(major,replace=True,n_samples=4961,random_state=66)
down_sample = pd.concat([minor,major_down])
down_sample["default.payment.next.month"].value_counts()

1    4998
0    4961
Name: default.payment.next.month, dtype: int64

In [54]:
x_down = down_sample[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6' ]]
y_down = down_sample["default.payment.next.month"]

In [60]:
model2 = Sequential()
model2.add(Dense(36, activation='relu', input_shape =(11,)))
model2.add(Dropout(0.3))
model2.add(Dense(36, activation='relu'))
model2.add(Dense(2, activation='softmax'))

model2.summary()

model2.compile(optimizer='adam',loss = "categorical_crossentropy",metrics=['accuracy'])



y_down_train_one = np_utils.to_categorical(y_down)
y_test_one = np_utils.to_categorical(y_test)
#y_down_train_one =np_utils.to_categorical(y_down)
model2.fit(x_down,y_down_train_one, epochs=20, validation_split=0.2,class_weight="balanced")
predict_two = model2.predict(x_test)
predict_int2 = np.argmax(predict_two, axis=1)




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 36)                432       
_________________________________________________________________
dropout_16 (Dropout)         (None, 36)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 36)                1332      
_________________________________________________________________
dense_48 (Dense)             (None, 2)                 74        
Total params: 1,838
Trainable params: 1,838
Non-trainable params: 0
_________________________________________________________________
Train on 7967 samples, validate on 1992 samples
Epoch 1/20
7967/7967 [==============================] - 1s 170us/step - loss: 6.0690 - acc: 0.6234 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/20
7967/7967 [==============================] - 1s 67us/step - loss: 6.0125

In [61]:
pre2 = pd.DataFrame(predict_int2)[0]
tru2= pd.DataFrame(y_test.values)[0]

In [62]:
score_matrix(tru2,pre2)

,Index,Score
0,Precision,0.218400
1,Recall,1.000000
2,FPR,1.000000
3,Accuracy,0.218400
4,F1,0.358503


## this time i don't know why my model predict it all 1 
## and i got recall 100% but its also useless